# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [14]:
useLogging = False # set to True to get logging information (and hopefully track which LLM is called when)
use_Ollama_For_Inference = True # set to True to use Ollama inference models (and pull at least the gemma:2b model)

import openai
#Defaults to OpenAI if use_Ollama_For_Inference=False and use_Ollama_For_Inference=False
openai.api_base = inferApiBase = embedApiBase =  "https://api.openai.com/v1"
openai.base_url = inferBaseUrl = embedBaseUrl = "https://api.openai.com"
openai.api_key = inferApiKey = embedApiKey = os.environ['OPENAI_API_KEY']
infer_model_name = "gpt-3.5-turbo"
llm_platform = "openai"


if use_Ollama_For_Inference:
    inferApiBase = "http://localhost:11434/v1"
    inferBaseUrl = "http://localhost:1143"
    inferApiKey = "ollama"
    infer_model_name = "gemma:7b" #you can/should customize this to test different Ollama LLMs. Use the NAME field from `ollama list`

print('Infer API Key:', inferApiKey)
print('Inference Model:', infer_model_name)

import langchain
if useLogging:
    import logging
    import requests 


    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s - %(levelname)s - %(filename)s:%(funcName)s:%(lineno)d - %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')
    requests.packages.urllib3.add_stderr_logger()
    OLLAMA_DEBUG=1
    langchain.debug = True
else:
    OLLAMA_DEBUG=0
    langchain.debug = False    

Infer API Key: ollama
Inference Model: gemma:7b


In [4]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [15]:
llm = ChatOpenAI(temperature=0.0, base_url=inferApiBase, model=infer_model_name)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [16]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Greetings, Andrew! 👋 It's wonderful to meet you. I'm running on a comprehensive dataset of human knowledge, so feel free to ask me anything that comes to mind. 😊"

In [17]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Greetings, Andrew! 👋 It's wonderful to meet you. I'm running on a comprehensive dataset of human knowledge, so feel free to ask me anything that comes to mind. 😊
Human: What is 1+1?
AI:

> Finished chain.


"1+1 equals 2. It's a basic arithmetic operation that represents the sum of two whole numbers."

In [18]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Greetings, Andrew! 👋 It's wonderful to meet you. I'm running on a comprehensive dataset of human knowledge, so feel free to ask me anything that comes to mind. 😊
Human: What is 1+1?
AI: 1+1 equals 2. It's a basic arithmetic operation that represents the sum of two whole numbers.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew. I retrieved this information from the context of our conversation.'

In [19]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Greetings, Andrew! 👋 It's wonderful to meet you. I'm running on a comprehensive dataset of human knowledge, so feel free to ask me anything that comes to mind. 😊
Human: What is 1+1?
AI: 1+1 equals 2. It's a basic arithmetic operation that represents the sum of two whole numbers.
Human: What is my name?
AI: Your name is Andrew. I retrieved this information from the context of our conversation.


In [20]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Greetings, Andrew! 👋 It's wonderful to meet you. I'm running on a comprehensive dataset of human knowledge, so feel free to ask me anything that comes to mind. 😊\nHuman: What is 1+1?\nAI: 1+1 equals 2. It's a basic arithmetic operation that represents the sum of two whole numbers.\nHuman: What is my name?\nAI: Your name is Andrew. I retrieved this information from the context of our conversation."}

In [21]:
memory = ConversationBufferMemory()

In [22]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [23]:
print(memory.buffer)

Human: Hi
AI: What's up


In [24]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [25]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [26]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [27]:
from langchain.memory import ConversationBufferWindowMemory

In [28]:
memory = ConversationBufferWindowMemory(k=1)               

In [29]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [30]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [22]:
llm = ChatOpenAI(temperature=0.0, base_url=inferApiBase, model=infer_model_name)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [23]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [23]:
conversation.predict(input="What is 1+1?")

'1+1 equals 2. Is there anything else you would like to know?'

In [24]:
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to ask?"

## ConversationTokenBufferMemory

In [ ]:
#!pip install tiktoken

In [24]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [25]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [26]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [32]:
from langchain.memory import ConversationSummaryBufferMemory


In [33]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [34]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI.'}

In [35]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [36]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI.
Human: What would be a good demo to show?
AI:

> Finished chain.


'For the meeting with the product team, a demo showcasing the latest features and improvements in our product would be great. For the LangChain project, a demo highlighting the progress made in developing the language processing algorithms would be impressive. And for the lunch meeting with the customer interested in AI, a demo showcasing how our AI technology can be customized to meet their specific needs would be ideal.'

In [37]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI.\nHuman: What would be a good demo to show?\nAI: For the meeting with the product team, a demo showcasing the latest features and improvements in our product would be great. For the LangChain project, a demo highlighting the progress made in developing the language processing algorithms would be impressive. And for the lunch meeting with the customer interested in AI, a demo showcasing how our AI technology can be customized to meet their specific needs would be ideal.'}